# Visualization and Modern Data Science

> Advanced Queries in SQL

Kuo, Yao-Jen <yaojenkuo@ntu.edu.tw> from [DATAINPOINT](https://www.datainpoint.com)

In [1]:
%LOAD sqlite3 db=../datasets/nba.db timeout=2 shared_cache=true

## Reclassifying Values with `CASE`

## The `CASE` statement is a conditional expression

Meaning it lets we add some "if this, then that..." logic to a query.

```sql
CASE WHEN condition_1 THEN result_1
     WHEN condition_2 THEN result_2
     ELSE result_else END AS variable_name
```

## We can use `CASE` in multiple ways

- Turning values into categories
- Turning existed categories into new categories

## Turning values into categories: heightMeters -> heightCategory

- Players taller than (or equal to) 2.0m
- Players shorter than 2.0m

In [2]:
SELECT firstName,
       lastName,
       heightMeters,
       CASE WHEN heightMeters >= 2 THEN 'Taller than 2 m'
            ELSE 'Shorter than 2 m' END AS heightCategory
  FROM players
 LIMIT 5;

firstName,lastName,heightMeters,heightCategory
LeBron,James,2.06,Taller than 2 m
Carmelo,Anthony,2.01,Taller than 2 m
Udonis,Haslem,2.03,Taller than 2 m
Dwight,Howard,2.08,Taller than 2 m
Andre,Iguodala,1.98,Shorter than 2 m


## Turning existing categories into new categories: pos -> posNew

The mapping rule for new positions:

```python
{
    'G-F': 'G',
    'G': 'G',
    'F-G': 'F',
    'F-C': 'F',
    'F': 'F',
    'C-F': 'C',
    'C': 'C'
}
```

In [3]:
SELECT firstName,
       lastName,
       pos,
       CASE WHEN pos IN ('G-F', 'G') THEN 'G'
            WHEN pos IN ('C-F', 'C') THEN 'C'
            ELSE 'F' END AS posNew
  FROM players
 LIMIT 5;

firstName,lastName,pos,posNew
LeBron,James,F,F
Carmelo,Anthony,F,F
Udonis,Haslem,F,F
Dwight,Howard,C-F,C
Andre,Iguodala,G-F,G


## Using Subqueries

## What is a sub-query?

A sub-query is nested inside another query, we just enclose the sub-query in parentheses and use it where needed.

## Filtering with subqueries in a `WHERE` clause

- Who is the top scorer in NBA?
- Who are the Lakers?

## Who is the top scorer in NBA?

- The first query selects from `career_summaries`
- The second query uses `MAX(points)` from previous query filtering `career_summaries`
- The third query uses `personId` from previous query filtering `players`

## The first query selects from `career_summaries`

In [4]:
SELECT MAX(points)
  FROM career_summaries;

MAX(points)
35236


## The second query uses `MAX(points)` from previous query filtering `career_summaries`

In [5]:
SELECT personId
  FROM career_summaries
 WHERE points = 35236;

personId
2544


## The third query uses `personId` from previous query filtering `players`

In [6]:
SELECT firstName,
       lastName
  FROM players
 WHERE personId = 2544;

firstName,lastName
LeBron,James


## Combining 3 queries into one to find out who the top scorer is

In [7]:
SELECT firstName,
       lastName
  FROM players
 WHERE personId = (
           SELECT personId
             FROM career_summaries
            WHERE points = (
                SELECT MAX(points)
                  FROM career_summaries
            )
       );

firstName,lastName
LeBron,James


## Who are the Lakers?

- The first query selects from `teams`
- The second query uses `teamId` from previous query filtering `players`

## The first query selects from `teams`

In [8]:
SELECT teamId
  FROM teams
 WHERE fullName = 'Los Angeles Lakers';

teamId
1610612747


## The second query uses `teamId` from previous query filtering `players`

In [9]:
SELECT firstName,
       lastName
  FROM players
 WHERE teamId = 1610612747;

firstName,lastName
LeBron,James
Jared,Dudley
Marc,Gasol
Wesley,Matthews
Markieff,Morris
Anthony,Davis
Dennis,Schroder
Kentavious,Caldwell-Pope
Montrezl,Harrell
Damian,Jones


## Combining 3 queries into one to find out who the Lakers are

In [10]:
SELECT firstName,
       lastName
  FROM players
 WHERE teamId = (SELECT teamId
                   FROM teams
                  WHERE nickname = 'Lakers');

firstName,lastName
LeBron,James
Jared,Dudley
Marc,Gasol
Wesley,Matthews
Markieff,Morris
Anthony,Davis
Dennis,Schroder
Kentavious,Caldwell-Pope
Montrezl,Harrell
Damian,Jones


## Generating columns with subqueries

Decomposing the NBA players based on their positions.

## Decomposing NBA players based on their positions

- The first query counts from `players`
- The second query summarizes and groups from `players`
- The third query derives percentage from `players`

## The first query counts from `players`

In [11]:
SELECT COUNT(*)
  FROM players;

COUNT(*)
493


## The second query summarizes and groups from `players`

In [12]:
SELECT pos,
       COUNT(*) player_count,
       493 AS ttl_player
  FROM players
 GROUP BY pos;

pos,player_count,ttl_player
C,34,493
C-F,27,493
F,131,493
F-C,43,493
F-G,23,493
G,182,493
G-F,53,493


## The third query derives percentage from `players`

In [13]:
SELECT pos,
       COUNT(*) player_count,
       493 AS ttl_player,
       COUNT(*) / 493.0 AS percentage
  FROM players
 GROUP BY pos;

pos,player_count,ttl_player,percentage
C,34,493,0.0689655172413793
C-F,27,493,0.0547667342799189
F,131,493,0.265720081135903
F-C,43,493,0.0872210953346856
F-G,23,493,0.0466531440162272
G,182,493,0.369168356997972
G-F,53,493,0.107505070993915


## Combining 3 queries into one to decompose NBA players based on their positions

In [14]:
SELECT pos,
       CAST(COUNT(*) AS REAL) / (SELECT COUNT(*)
                                   FROM players) AS ratio
  FROM players
 GROUP BY pos;

pos,ratio
C,0.0689655172413793
C-F,0.0547667342799189
F,0.265720081135903
F-C,0.0872210953346856
F-G,0.0466531440162272
G,0.369168356997972
G-F,0.107505070993915


## Creating Views

## What are views?

> In a database, a view is the result set of a stored query on the data, which the database users can query just as they would in a persistent database collection object. This pre-established query command is kept in the database dictionary.

Source: <https://en.wikipedia.org/wiki/View_(SQL)>

## Views are like

- A persistent sub-query.
- A stored `SELECT` statement that feels like a table for users.
- A feature lies between subqueries and creating a table.

## Using `CREATE VIEW` to create a view

```sql
CREATE VIEW view_name (column_names)
    AS
SELECT statement;
```

## List the top 5 NBA scorers based on `points`

In [15]:
DROP VIEW IF EXISTS top_five_scorers;

In [16]:
CREATE VIEW top_five_scorers (personId, points)
AS
SELECT personId,
       points
  FROM career_summaries
 ORDER BY points DESC
 LIMIT 5;

## Once a view is created, use it like a table

In [17]:
SELECT * FROM top_five_scorers;

personId,points
2544,35236
2546,26980
201142,23491
201935,21853
201566,21095


## Who are the top 5 NBA scorers?

In [18]:
SELECT firstName || ' ' || lastName AS fullName
  FROM players
 WHERE personId IN (SELECT personId
                      FROM top_five_scorers);

fullName
LeBron James
Carmelo Anthony
Kevin Durant
Russell Westbrook
James Harden


## Joining Tables in a Releational Database

## What is a relational database?

> An application that supports data stored across multiple, related tables. In a relational model, each table typically holds data on one entity and each row in the table describes one of those entities. A process known as a table join allows us to link rows in one table to rows in other tables.

## Why relational database?

> Using the relational model, we can build tables that eliminate duplicate data, are easier to maintain, and provide for increased flexibility in writing queries to get just the data we want.

## Linking tables using `JOIN...ON...` clause

The query examines both tables and then returns columns from both tables where the values match in the columns specified in the `ON` clause.

```sql
SELECT *
  FROM left_table 
  JOIN right_table
    ON left_table.join_key = right_table.join_key
```

## Linking `nba.players` with `nba.careerSummaries`

```sql
SELECT *
  FROM nba.players
  JOIN nba.careerSummaries
    ON players.personId = careerSummaries.personId;
```

In [19]:
SELECT *
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 LIMIT 5;

firstName,lastName,temporaryDisplayName,personId,teamId,jersey,pos,heightFeet,heightInches,heightMeters,weightPounds,weightKilograms,dateOfBirthUTC,nbaDebutYear,yearsPro,collegeName,lastAffiliation,country,personId,tpp,ftp,fgp,ppg,rpg,apg,bpg,mpg,spg,assists,blocks,steals,turnovers,offReb,defReb,totReb,fgm,fga,tpm,tpa,ftm,fta,pFouls,points,gamesPlayed,gamesStarted,plusMinus,min,dd2,td3
LeBron,James,"James, LeBron",2544,1610612747,23,F,6,9,2.06,250,113.4,1984-12-30,2003,17,St. Vincent-St. Mary HS (OH),St. Vincent-St. Mary HS (OH)/USA,USA,2544,34.5,73.4,50.4,27,7.5,7.4,0.8,38.3,1.6,9659,980,2057,4572,1532,8187,9719,12852,25508,1968,5708,7564,10312,2388,35236,1304,1303,7128,49890,507,99
Carmelo,Anthony,"Anthony, Carmelo",2546,1610612757,0,F,6,7,2.01,238,108,1984-05-29,2003,17,Syracuse,Syracuse/USA,USA,2546,35,81.3,44.7,23.3,6.4,2.9,0.5,35.4,1,3317,584,1165,2973,1912,5522,7434,9661,21596,1516,4327,6142,7557,3317,26980,1160,1117,1545,41014,172,2
Udonis,Haslem,"Haslem, Udonis",2617,1610612748,40,F,6,8,2.03,235,106.6,1980-06-09,2003,17,Florida,Florida/USA,USA,2617,8.6,75.5,48.9,7.6,6.7,0.8,0.3,25.1,0.5,729,249,422,804,1607,4147,5754,2643,5400,5,58,1232,1631,2043,6523,858,500,489,21563,134,0
Dwight,Howard,"Howard, Dwight",2730,1610612755,39,C-F,6,10,2.08,265,120.2,1985-12-08,2004,16,SW Atlanta Christian Academy (GA),SW Atlanta Christian Academy (GA)/USA,USA,2730,15.9,56.5,58.6,16.4,12.1,1.4,1.9,32.9,0.9,1610,2169,1035,3213,3946,10066,14012,6844,11673,13,82,5205,9212,3722,18906,1154,1049,3015,37983,737,1
Andre,Iguodala,"Iguodala, Andre",2738,1610612748,28,G-F,6,6,1.98,215,97.5,1984-01-28,2004,16,Arizona,Arizona/USA,USA,2738,33.3,70.8,46.4,11.8,5,4.2,0.5,32.8,1.5,4952,590,1715,2174,1077,4771,5848,4941,10644,1045,3135,2805,3960,2097,13732,1167,781,2899,38267,81,9


## Relating tables with key columns: Primary key

One or more columns whose values uniquely identify each row in a table.

1. Each column in the key must have a unique value for each row
2. No column in the key can have missing values

## Relating tables with key columns: Foreign key

One or more columns in a table that match the primary key of another table. Foreign key ensures that we don’t end up with rows in one table that have no relation to rows in the other tables we can join them to.

## We can examine key columns in metadata if any

In [20]:
SELECT *
  FROM PRAGMA_TABLE_INFO('career_summaries')
 WHERE pk = 1;

cid,name,type,notnull,dflt_value,pk
0,personId,INTEGER,0,NULL,1


In [21]:
SELECT *
  FROM PRAGMA_TABLE_INFO('teams')
 WHERE pk = 1;

cid,name,type,notnull,dflt_value,pk
6,teamId,INTEGER,0,NULL,1


In [22]:
SELECT *
  FROM PRAGMA_TABLE_INFO('players')
 WHERE pk = 1;

cid,name,type,notnull,dflt_value,pk
3,personId,INTEGER,0,NULL,1


## In contrast to `JOIN`, the `LEFT JOIN` keyword returns all rows from the left table and display blank rows from the other table if no matching values are found

## Creating a left table using via `CREATE VIEW`

Let's called the left table as `veteran_players`.

In [23]:
DROP VIEW IF EXISTS veteran_players;

In [24]:
CREATE VIEW veteran_players (personId, temporaryDisplayName)
AS
SELECT personId,
       temporaryDisplayName
  FROM players
 LIMIT 15;

## Creating a right table also via `CREATE VIEW`

In [25]:
DROP VIEW IF EXISTS top_scorers;

In [26]:
CREATE VIEW top_scorers (personId, ppg)
    AS
SELECT personId,
       ppg
  FROM career_summaries
 ORDER BY ppg DESC
 LIMIT 15;

## The default `JOIN` leaves us the top 2 rookies in 2003 draft

In [27]:
SELECT *
  FROM veteran_players
  JOIN top_scorers
    ON veteran_players.personId = top_scorers.personId;

personId,temporaryDisplayName,personId,ppg
2544,"James, LeBron",2544,27
2546,"Anthony, Carmelo",2546,23.3


## The top 3 drafts in 2003

![Imgur](https://i.imgur.com/kAjBFy1.jpg)

Source: Google Search

## The `LEFT JOIN` leaves us all the veteran players

In [28]:
SELECT *
  FROM veteran_players
  LEFT JOIN top_scorers
    ON veteran_players.personId = top_scorers.personId;

personId,temporaryDisplayName,personId,ppg
2544,"James, LeBron",2544,27
2546,"Anthony, Carmelo",2546,23.3
2617,"Haslem, Udonis",NULL,NULL
2730,"Howard, Dwight",NULL,NULL
2738,"Iguodala, Andre",NULL,NULL
2772,"Ariza, Trevor",NULL,NULL
101108,"Paul, Chris",NULL,NULL
101141,"Ilyasova, Ersan",NULL,NULL
101150,"Williams, Lou",NULL,NULL
200746,"Aldridge, LaMarcus",NULL,NULL


## Using `IS NULL` to find rows with missing values

Veterans still play, but no longer top scorers.

In [29]:
SELECT veteran_players.personId,
       veteran_players.temporaryDisplayName
  FROM veteran_players
  LEFT JOIN top_scorers
    ON veteran_players.personId = top_scorers.personId
 WHERE top_scorers.ppg IS NULL;

personId,temporaryDisplayName
2617,"Haslem, Udonis"
2730,"Howard, Dwight"
2738,"Iguodala, Andre"
2772,"Ariza, Trevor"
101108,"Paul, Chris"
101141,"Ilyasova, Ersan"
101150,"Williams, Lou"
200746,"Aldridge, LaMarcus"
200752,"Gay, Rudy"
200755,"Redick, JJ"


## Joining tables involves understanding how the database designer intends for the tables to relate, also known as the database’s relational model

## The three types of table relationships are straight-forward

- one to one
- one to many
- many to many

## One-to-One Relationship

There is only one match for each of the two tables. In addition, there are no duplicate id values in either table, say the relationship between active players and their career summaries.

In [30]:
SELECT *
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 LIMIT 5;

firstName,lastName,temporaryDisplayName,personId,teamId,jersey,pos,heightFeet,heightInches,heightMeters,weightPounds,weightKilograms,dateOfBirthUTC,nbaDebutYear,yearsPro,collegeName,lastAffiliation,country,personId,tpp,ftp,fgp,ppg,rpg,apg,bpg,mpg,spg,assists,blocks,steals,turnovers,offReb,defReb,totReb,fgm,fga,tpm,tpa,ftm,fta,pFouls,points,gamesPlayed,gamesStarted,plusMinus,min,dd2,td3
LeBron,James,"James, LeBron",2544,1610612747,23,F,6,9,2.06,250,113.4,1984-12-30,2003,17,St. Vincent-St. Mary HS (OH),St. Vincent-St. Mary HS (OH)/USA,USA,2544,34.5,73.4,50.4,27,7.5,7.4,0.8,38.3,1.6,9659,980,2057,4572,1532,8187,9719,12852,25508,1968,5708,7564,10312,2388,35236,1304,1303,7128,49890,507,99
Carmelo,Anthony,"Anthony, Carmelo",2546,1610612757,0,F,6,7,2.01,238,108,1984-05-29,2003,17,Syracuse,Syracuse/USA,USA,2546,35,81.3,44.7,23.3,6.4,2.9,0.5,35.4,1,3317,584,1165,2973,1912,5522,7434,9661,21596,1516,4327,6142,7557,3317,26980,1160,1117,1545,41014,172,2
Udonis,Haslem,"Haslem, Udonis",2617,1610612748,40,F,6,8,2.03,235,106.6,1980-06-09,2003,17,Florida,Florida/USA,USA,2617,8.6,75.5,48.9,7.6,6.7,0.8,0.3,25.1,0.5,729,249,422,804,1607,4147,5754,2643,5400,5,58,1232,1631,2043,6523,858,500,489,21563,134,0
Dwight,Howard,"Howard, Dwight",2730,1610612755,39,C-F,6,10,2.08,265,120.2,1985-12-08,2004,16,SW Atlanta Christian Academy (GA),SW Atlanta Christian Academy (GA)/USA,USA,2730,15.9,56.5,58.6,16.4,12.1,1.4,1.9,32.9,0.9,1610,2169,1035,3213,3946,10066,14012,6844,11673,13,82,5205,9212,3722,18906,1154,1049,3015,37983,737,1
Andre,Iguodala,"Iguodala, Andre",2738,1610612748,28,G-F,6,6,1.98,215,97.5,1984-01-28,2004,16,Arizona,Arizona/USA,USA,2738,33.3,70.8,46.4,11.8,5,4.2,0.5,32.8,1.5,4952,590,1715,2174,1077,4771,5848,4941,10644,1045,3135,2805,3960,2097,13732,1167,781,2899,38267,81,9


## One-to-Many Relationship

A key value in the first table will have multiple matching values in the second table’s joined column, say the relationship between a team and its roster.

In [31]:
SELECT *
  FROM teams
  JOIN players
    ON teams.teamId = players.teamId
 LIMIT 5;

isNBAFranchise,isAllStar,city,altCityName,fullName,tricode,teamId,nickname,urlName,teamShortName,confName,divName,firstName,lastName,temporaryDisplayName,personId,teamId,jersey,pos,heightFeet,heightInches,heightMeters,weightPounds,weightKilograms,dateOfBirthUTC,nbaDebutYear,yearsPro,collegeName,lastAffiliation,country
1,0,Los Angeles,Los Angeles Lakers,Los Angeles Lakers,LAL,1610612747,Lakers,lakers,L.A. Lakers,West,Pacific,LeBron,James,"James, LeBron",2544,1610612747,23,F,6,9,2.06,250,113.4,1984-12-30,2003,17,St. Vincent-St. Mary HS (OH),St. Vincent-St. Mary HS (OH)/USA,USA
1,0,Portland,Portland,Portland Trail Blazers,POR,1610612757,Trail Blazers,blazers,Portland,West,Northwest,Carmelo,Anthony,"Anthony, Carmelo",2546,1610612757,0,F,6,7,2.01,238,108,1984-05-29,2003,17,Syracuse,Syracuse/USA,USA
1,0,Miami,Miami,Miami Heat,MIA,1610612748,Heat,heat,Miami,East,Southeast,Udonis,Haslem,"Haslem, Udonis",2617,1610612748,40,F,6,8,2.03,235,106.6,1980-06-09,2003,17,Florida,Florida/USA,USA
1,0,Philadelphia,Philadelphia,Philadelphia 76ers,PHI,1610612755,76ers,sixers,Philadelphia,East,Atlantic,Dwight,Howard,"Howard, Dwight",2730,1610612755,39,C-F,6,10,2.08,265,120.2,1985-12-08,2004,16,SW Atlanta Christian Academy (GA),SW Atlanta Christian Academy (GA)/USA,USA
1,0,Miami,Miami,Miami Heat,MIA,1610612748,Heat,heat,Miami,East,Southeast,Andre,Iguodala,"Iguodala, Andre",2738,1610612748,28,G-F,6,6,1.98,215,97.5,1984-01-28,2004,16,Arizona,Arizona/USA,USA


## Joining tables is like concatenating tables horizontally

![Imgur](https://i.imgur.com/hq7fS67.png)

Source: [Pandas User Guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

## We can also concatenating tables vertically via `UNION` clause

![Imgur](https://i.imgur.com/B7xawvp.png)

Source: [Pandas User Guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [32]:
SELECT firstName,
       lastName,
       'height' AS category,
       heightmeters AS value
  FROM players
 WHERE firstName = 'LeBron'
 UNION
SELECT firstName,
       lastName,
       'weight' AS category,
       weightKilograms AS value
  FROM players
 WHERE firstName = 'LeBron';

firstName,lastName,category,value
LeBron,James,height,2.06
LeBron,James,weight,113.4
